In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import glob
import scipy
import copy
import math
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker

#plotting with Cartopy. 
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import cm
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import ticker


# import scipy
from datetime import datetime
import os
# import utils
import importlib


# import statsmodels.api as sm
# from windspharm.xarray import VectorWind
# from windspharm.standard import VectorWind
# from windspharm.examples import example_data_path
# from windspharm.tools import prep_data, recover_data, order_latdim
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from shapely.geometry.polygon import LinearRing
from dask.diagnostics import ProgressBar
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as mpatches
import math
import statsmodels.api as sm
from statsmodels.graphics import tsaplots
import geocat.comp as gcomp
import shutil
import metpy.calc as mpcalc
import numba as nb
from numba import njit, prange

import xesmf as xe


# Experiments: 
    -Free Running: 
        -/glade/campaign/cisl/aiml/wchapman/CAM_runs/f.e21.DAcompset.f09_d025_free_MJO_1982
    -Nudge No Stochai: 
        -/glade/scratch/wchapman/archive/f.e21.DAcompset.f09_d025_Seasonal_stochai_UV_00_1982
    -Nudge Stochai
        -/glade/scratch/wchapman/archive/f.e21.DAcompset.f09_d025_Seasonal_stochai_UV_05_1982_MJO_v3
    -DA No Stochai: 
        -/glade/scratch/wchapman/archive/f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_00_1982
    -DA Stochai
        -/glade/scratch/wchapman/archive/f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_05_1982/

In [2]:
if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')

client does not exist yet


In [3]:
from distributed import Client
from ncar_jobqueue import NCARCluster

cluster = NCARCluster(project='P54048000',walltime='06:00:00')
cluster.scale(40)
client = Client(cluster)
client

/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/apps/opt/conda/envs/npl-2023a/lib/python3.9/site-packages/dask_jobqueue/core.py:274: FutureWarning: env_extra has been renamed to job_script_prologue. You are still using it (even if only set to []; please also check config files). If you did not set job_script_prologue yet, env_extra will be respected for now, but it will be removed in a future release. If you already set job_script_prologue, env_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/apps/opt/conda

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dango/proxy/44433/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dango/proxy/44433/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.20:38369,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dango/proxy/44433/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Define and Go

In [22]:
lev=500
modvers_comp = 'f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_03_1982_r1'
fil_out_comp = '/glade/scratch/wchapman/ADF/' + modvers_comp +'/ts/' + modvers_comp + '.h1.UV'+str(lev)+'.198201-201012.nc'
path_tofiles_comp ='/glade/scratch/wchapman/archive/' + modvers_comp + '/atm/hist/*h1*.nc'
remake_files_comp = False

In [10]:
lev=500
modvers_comp = 'f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_00_1982'
fil_out_comp = '/glade/scratch/wchapman/ADF/' + modvers_comp +'/ts/' + modvers_comp + '.h1.UV'+str(lev)+'.198201-201012.nc'
path_tofiles_comp ='/glade/scratch/wchapman/archive/' + modvers_comp + '/atm/hist/*h1*.nc'
remake_files_comp = False

In [5]:
lev=500
modvers_comp = 'ERA_uv258_'
fil_out_comp = '/glade/scratch/wchapman/ADF/'+ modvers_comp + '.h1.UV'+str(lev)+'.198201-201012.nc'
path_tofiles_comp ='/glade/scratch/wchapman/TEMPERAI/' + modvers_comp + '*.nc'
remake_files_comp = False

In [10]:
lev=200
modvers_comp = 'f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_03_1982_r1'
fil_out_comp = '/glade/scratch/wchapman/ADF/' + modvers_comp +'/ts/' + modvers_comp + '.h1.STOCH'+str(lev)+'.198201-201012.nc'
path_tofiles_comp ='/glade/campaign/cisl/aiml/wchapman/CAM_runs/Nudge_DA/' + modvers_comp + '/atm/hist/*h1*.nc'
remake_files_comp = False

## Functions

In [11]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


def wgt_rmse(fld1, fld2, wgt):
    """Calculated the area-weighted RMSE.
    Inputs are 2-d spatial fields, fld1 and fld2 with the same shape.
    They can be xarray DataArray or numpy arrays.
    Input wgt is the weight vector, expected to be 1-d, matching length of one dimension of the data.
    Returns a single float value.
    """
    assert len(fld1.shape) == 2,     "Input fields must have exactly two dimensions."
    assert fld1.shape == fld2.shape, "Input fields must have the same array shape."
    # in case these fields are in dask arrays, compute them now.
    if hasattr(fld1, "compute"):
        fld1 = fld1.compute()
    if hasattr(fld2, "compute"):
        fld2 = fld2.compute()
    if isinstance(fld1, xr.DataArray) and isinstance(fld2, xr.DataArray):
        return (np.sqrt(((fld1 - fld2)**2).weighted(wgt).mean())).values.item()
    else:
        check = [len(wgt) == s for s in fld1.shape]
        if ~np.any(check):
            raise IOError(f"Sorry, weight array has shape {wgt.shape} which is not compatible with data of shape {fld1.shape}")
        check = [len(wgt) != s for s in fld1.shape]
        dimsize = fld1.shape[np.argwhere(check).item()]  # want to get the dimension length for the dim that does not match the size of wgt
        warray = np.tile(wgt, (dimsize, 1)).transpose()   # May need more logic to ensure shape is correct.
        warray = warray / np.sum(warray) # normalize
        wmse = np.nansum(warray * (fld1 - fld2)**2)
        return np.sqrt( wmse ).item()

#######


def calculate_h1_data(path_tofiles, fil_out,dovars=['U','V'],remake_files=False,lev=300):
    """
    Calculate h1 data for given files.

    Parameters:
    path_tofiles (str): Path to the files.
    fil_out (str): Path to the output file for chunked data.
    fil_out_blocked (str): Path to the output file for blocked data.
    remake_files (bool, optional): Whether to remake the output files if they exist. Defaults to False.

    Returns:
    blocked_data (xarray.Dataset): The blocked data.
    bbedo_chunked (xarray.Dataset): The chunked data.
    """
    
    # List of variables to use
    if not isinstance(dovars, list):
        dovars = [dovars]
    # Remove output file if it exists and remake_files is True
    if (os.path.exists(fil_out)) and (remake_files):
        os.remove(fil_out)
    
    # If output file does not exist, create chunked data and save it
    if not os.path.exists(fil_out):
        fns = sorted(glob.glob(path_tofiles))
        program_starts = time.time()
        for ee, group in enumerate(chunker(fns, 30)):
            print(f'doing group: {ee} of {len(fns) / 30}')
            DS = xr.open_mfdataset(group, parallel=True)
            
            if all(np.isin(['longitude','latitude','time','level'],DS.coords)):
                DS = DS.rename({'longitude':'lon','latitude':'lat','level':'lev','u':'U','v':'V'})
            
            DS = DS.sel(lev=lev,method='nearest')
            
            if not xr.infer_freq(DS['time'])=='D':
                print('freq is not daily...resampling')
                DS = DS.resample(time='1D').mean()
                
            print('..opened..')
            if ee == 0:
                # bbedo = DS[dovars].load()
                bbedo = DS[dovars]
            else: 
                # temp_bbedo = DS[dovars].load()
                temp_bbedo = DS[dovars]
                print('concat')
                bbedo = xr.concat([bbedo,temp_bbedo], dim='time')
            now = time.time()
            print(f"It has been {now - program_starts} seconds since the loop started")
        bbedo_chunked = bbedo.chunk({'time': 100, 'lat': 192, 'lon': 288})
        print('...saving...')
        bbedo_chunked.to_netcdf(fil_out)
    else:
        print('file existed... opening')
        bbedo = xr.open_dataset(fil_out)
    print('.... done done done....')
    # bbedo_chunked.load()

## Calculate then ReGrid

In [12]:
calculate_h1_data(path_tofiles=path_tofiles_comp,fil_out=fil_out_comp,dovars=['Nudge_U','Nudge_V','Stochai_U','Stochai_V'],remake_files=False,lev=lev)

doing group: 0 of 117.7
..opened..
It has been 1.7945899963378906 seconds since the loop started
doing group: 1 of 117.7
..opened..
concat
It has been 3.8325083255767822 seconds since the loop started
doing group: 2 of 117.7
..opened..
concat
It has been 6.032780647277832 seconds since the loop started
doing group: 3 of 117.7
..opened..
concat
It has been 7.896076679229736 seconds since the loop started
doing group: 4 of 117.7
..opened..
concat
It has been 9.869661331176758 seconds since the loop started
doing group: 5 of 117.7
..opened..
concat
It has been 12.767354488372803 seconds since the loop started
doing group: 6 of 117.7
..opened..
concat
It has been 14.630245923995972 seconds since the loop started
doing group: 7 of 117.7
..opened..
concat
It has been 16.546322345733643 seconds since the loop started
doing group: 8 of 117.7
..opened..
concat
It has been 19.06063437461853 seconds since the loop started
doing group: 9 of 117.7
..opened..
concat
It has been 21.095407962799072 se

## Regrid and Save

In [52]:
# open file
fn = fil_out_comp
# fn = '/glade/scratch/wchapman/ADF/f.e21.DAcompset.f09_d025_Seasonal_stochai_UV_05_1982_MJO_v3/ts/f.e21.DAcompset.f09_d025_Seasonal_stochai_UV_05_1982_MJO_v3.h1.UV.198201-201012.nc'

In [21]:
%%time 
#grab an example file to get the grid: 
fns = sorted(glob.glob('/glade/campaign/cisl/aiml/wchapman/20CSeasForecast/Daily/Velocity_Potential/Daily_Feb_VelocityPotential_*.nc'))
DScamgrid = xr.open_dataset(fns[0])
DScamgrid

#set lat/lon to regrid to: 
latcam = DScamgrid.latitude
loncam = DScamgrid.longitude

#make regridder structure:
ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.array(DScamgrid.latitude)),
        "lon": (["lon"], np.array(DScamgrid.longitude)),
    }
)
#open the GPCP file: 
fn = fil_out_comp
DSall = xr.open_dataset(fn)
# ##build regridder method: 
regridder = xe.Regridder(DSall, ds_out, "bilinear")
# #regrid. 
ds_out = regridder(DSall) #this is an xarray instance now. 
#compute from xarray in dask 
#save:

fn_regrid=fil_out_comp.split('UV'+str(lev))[0]+'UV'+str(lev)+'.ERAgrid'+fil_out_comp.split('UV'+str(lev))[-1]

print('saving... ',fn_regrid)
ds_out.to_netcdf(fn_regrid)

saving...  /glade/scratch/wchapman/ADF/f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_03_1982_r1/ts/f.e21.DAcompset.f09_d025_Seasonal_DA_stochai_UV_03_1982_r1.h1.UV500.ERAgrid.198201-201012.nc
CPU times: user 17.1 s, sys: 2.78 s, total: 19.9 s
Wall time: 31 s
